In [98]:
import numpy as np
from aocd import get_data, submit 
import re
data = """        ...#    
        .#..    
        #...    
        ....    
...#.......#    
........#...    
..#....#....    
..........#.    
        ...#....
        .....#..
        .#......
        ......#.

10R5L5R10L4R5L5"""
data = get_data(year=2022, day=22)
field, instructions = data.split("\n\n")
field = field.splitlines()
width = max([len(line) for line in field])

field = [line + " " * (width - len(line)) for line in field]

field = np.array(list(map(list, field)))

directions = list(zip(re.split("[LR]+", instructions), re.split("\d+", instructions)))

In [99]:
starting_x = np.where(field[0,:] == ".")[0][0]
starting_y = 0
starting_x

50

In [101]:
max_y, max_x = field.shape

In [102]:
headings = ["N", "E", "S", "W"]

get_facing_idx = lambda facing: next(i for i, f in enumerate(headings) if f == facing)
turn_dir = {"L": -1, "R": 1, "": 0}
get_new_facing = lambda facing, turn: headings[ (get_facing_idx(facing) + turn_dir[turn]) % len(headings) ]


In [103]:
list(headings)

['N', 'E', 'S', 'W']

In [113]:
move_dict = {
    "N": (-1, 0),
    "S": (1, 0),
    "E": (0, 1),
    "W": (0, -1)
}

def find_wrap_topdown(x, y, facing):
    dy, dx = move_dict[facing]
    new_x = (x + dx) % max_x
    new_y = (y + dy) % max_y
    while field[new_y, new_x] == ' ':
        new_x = (new_x + dx) % max_x
        new_y = (new_y + dy) % max_y
    return new_x, new_y, facing
    

def move_until_stops(x, y, dist, facing, find_wrap):
    dist = int(dist)
    dy, dx = move_dict[facing]
    print(f"Moving y:{dy}, x:{dx} ({facing})")
    for _ in range(dist):
        new_x = (x + dx) % max_x
        new_y = (y + dy) % max_y
        new_facing = facing
        if field[new_y, new_x] == ' ':
            new_x, new_y, new_facing = find_wrap(x, y, facing)
        if field[new_y, new_x] == "#":
            print(f"Wall at ({new_x}, {new_y})")
            break
        if field[new_y, new_x] == ".":
            x = new_x
            y = new_y
            facing = new_facing
            print(f"Moving to ({x}, {y})")
        else:
            raise ValueError(field[new_y, new_x])

    return x, y, facing

In [114]:
def simulate_path(x, y, facing):
    for dist, turn in directions:

        facing = get_new_facing(facing, turn)
        print(f"Turning {turn}, now facing {facing}")
        print(f"Will move {dist}")

        x, y, facing = move_until_stops(x, y, dist, facing, find_wrap_topdown)

        print(f"Now at ({x}, {y}) facing:{facing}")
    return x, y

x, y = simulate_path(starting_x, starting_y, "E")

Turning , now facing E
Will move 25
Moving y:0, x:1 (E)
Moving to (51, 0)
Moving to (52, 0)
Moving to (53, 0)
Moving to (54, 0)
Moving to (55, 0)
Moving to (56, 0)
Moving to (57, 0)
Moving to (58, 0)
Moving to (59, 0)
Moving to (60, 0)
Moving to (61, 0)
Moving to (62, 0)
Wall at (63, 0)
Now at (62, 0) facing:E
Turning L, now facing N
Will move 42
Moving y:-1, x:0 (N)
Moving to (62, 149)
Moving to (62, 148)
Moving to (62, 147)
Moving to (62, 146)
Moving to (62, 145)
Moving to (62, 144)
Moving to (62, 143)
Moving to (62, 142)
Moving to (62, 141)
Wall at (62, 140)
Now at (62, 141) facing:N
Turning L, now facing W
Will move 39
Moving y:0, x:-1 (W)
Moving to (61, 141)
Moving to (60, 141)
Moving to (59, 141)
Moving to (58, 141)
Moving to (57, 141)
Moving to (56, 141)
Moving to (55, 141)
Wall at (54, 141)
Now at (55, 141) facing:W
Turning L, now facing S
Will move 15
Moving y:1, x:0 (S)
Moving to (55, 142)
Moving to (55, 143)
Moving to (55, 144)
Moving to (55, 145)
Wall at (55, 146)
Now at (5

In [115]:
answer = (y+1) * 1000 + (x+1) * 4 + (get_facing_idx(facing) - 1) % len(headings)
answer

97356

In [122]:
field[51, 49]

' '

In [128]:
face_ids = {
    (0,0): 5,
    (0,1): 1,
    (0,2): 2,
    (0,3): 4,
    (1,0): 5,
    (1,1): 3,
    (1,2): set([3,2]),
    (2,0): set([4,5]),
    (2,1): 4,
    (2,2): 2,
    (3,-1): 3,
    (3,0): 5,
    (3,1): 6, 
    (3,2): 2
}

N = 50
get_face_id = lambda x, y: face_ids[(x//N, y//N)]

In [127]:
face_rotations = {
    (2,3):
    (2,4):
    (2,6):
    (3,2):
    (4,2):
    (6,2):
    (1,5):
    (3,5):
    (4,5):
    (5,1):
    (5,3):
    (5,4):
}

1